# 0. 들어가기 앞서

* 고객 질문: 1, 상담원 질문: 2, 고객 및 상담원 대답: 0

# 1. 라이브러리 로드

In [1]:
import sys
sys.version

'3.8.5 (default, Sep  4 2020, 07:30:14) \n[GCC 7.3.0]'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup ##

In [5]:
##GPU 사용 시
device = torch.device("cuda:0")

# 2. 모델, 사전, 데이터셋 불러오기

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/adminuser/notebooks/modeling/question/[0차] KoBERT_QA_baseline/.cache/kobert_v1.zip
using cached model. /home/adminuser/notebooks/modeling/question/[0차] KoBERT_QA_baseline/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
import os
os.listdir("../[0차] 원본_preprocessing/")

['.ipynb_checkpoints',
 '질의응답_K쇼핑_질문유형분류_원본.csv',
 '[0차] 원본데이터_preprocessing.ipynb',
 '질의응답_K쇼핑_질문분류_원본.csv']

In [8]:
df=pd.read_csv("../[0차] 원본_preprocessing/질의응답_K쇼핑_질문분류_원본.csv", index_col=0)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# 3. 질문분류시작

* 고객 질문: 1, 상담원 질문: 2, 고객 및 상담원 대답: 0

In [9]:
# 2중 리스트로 변환됨

data_list = []
for q, label in zip(df["msg"],df["QA"])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

## 3-1. Train / Test set 분리

* 라벨링은 이미 진행했으므로, 바로 train/ test 분리 진행

In [10]:
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [11]:
print(len(dataset_train))
print(len(dataset_test))

753924
251309


## 3-2. KoBERT 입력 데이터로 만들기

* 데이터를 train data와 test data로 나누었다면 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 해주어야 한다

In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
# Setting parameters

max_len = 32 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [14]:
# 토큰화
tokenizer= get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/adminuser/notebooks/modeling/question/[0차] KoBERT_QA_baseline/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
# 첫 번째는 패딩된 시퀀스
# 두 번째는 길이와 타입에 대한 내용
# 세 번재는 어텐션 마스크 시퀀스

data_train[0]

(array([   2, 2847, 4103, 5130,  793, 5925,  517,   54, 2926, 6141, 6050,
        2822, 5330, 7287,  517, 7707, 7494,  517, 7710, 7753, 6664,  517,
        6539, 5931, 3647, 6314, 2650, 6749, 6964, 4227, 1767,    3],
       dtype=int32),
 array(32, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 2)

In [16]:
from transformers import AutoModel, AutoTokenizer

In [17]:
# pytorch용 DataLoader 사용(torch 형식의 dataset을 만들어주기)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## 3-3. KoBERT 학습모델 만들기

* 고객 질문: 1, 상담원 질문: 2, 고객 및 상담원 대답: 0 
* 3가지의 class를 분류하기 때문에 num_classes는 3으로 입력

In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) ##

In [19]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## 3-4. KoBERT 모델 학습시키기

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_14730/2673822008.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/23561 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6713322997093201 train acc 0.8125
epoch 1 batch id 201 loss 0.37738046050071716 train acc 0.8468594527363185
epoch 1 batch id 401 loss 0.22280533611774445 train acc 0.8641677057356608
epoch 1 batch id 601 loss 0.33898356556892395 train acc 0.8824875207986689
epoch 1 batch id 801 loss 0.23097948729991913 train acc 0.8920099875156055
epoch 1 batch id 1001 loss 0.167324960231781 train acc 0.8983204295704296
epoch 1 batch id 1201 loss 0.08900411427021027 train acc 0.9041944213155704
epoch 1 batch id 1401 loss 0.12853601574897766 train acc 0.908168272662384
epoch 1 batch id 1601 loss 0.23934578895568848 train acc 0.9110126483447845
epoch 1 batch id 1801 loss 0.27416399121284485 train acc 0.9137978900610771
epoch 1 batch id 2001 loss 0.18008917570114136 train acc 0.916104447776112
epoch 1 batch id 2201 loss 0.1914428323507309 train acc 0.9173671058609723
epoch 1 batch id 2401 loss 0.13977640867233276 train acc 0.9189139941690962
epoch 1 batch id 2601 loss 0.20029728

epoch 1 batch id 21601 loss 0.059612151235342026 train acc 0.9372728693116059
epoch 1 batch id 21801 loss 0.10626745223999023 train acc 0.9373236892803083
epoch 1 batch id 22001 loss 0.032149143517017365 train acc 0.9373863688014181
epoch 1 batch id 22201 loss 0.416692852973938 train acc 0.937436658258637
epoch 1 batch id 22401 loss 0.3622431457042694 train acc 0.9374665193518147
epoch 1 batch id 22601 loss 0.3928498923778534 train acc 0.9375221229149153
epoch 1 batch id 22801 loss 0.19935035705566406 train acc 0.9375849743432305
epoch 1 batch id 23001 loss 0.06947287172079086 train acc 0.9376290704751967
epoch 1 batch id 23201 loss 0.1982797086238861 train acc 0.9376589371147795
epoch 1 batch id 23401 loss 0.1719861626625061 train acc 0.9376976411264476
epoch 1 train acc 0.9377612898433852


/tmp/ipykernel_14730/2673822008.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



epoch 5 batch id 22401 loss 0.19471345841884613 train acc 0.9553856412660149
epoch 5 batch id 22601 loss 0.14114578068256378 train acc 0.9554444493606478
epoch 5 batch id 22801 loss 0.21647141873836517 train acc 0.9554720735932635
epoch 5 batch id 23001 loss 0.0890384167432785 train acc 0.9554924242424242
epoch 5 batch id 23201 loss 0.13988149166107178 train acc 0.955494914012327
epoch 5 batch id 23401 loss 0.05011158064007759 train acc 0.9555173924191274
epoch 5 train acc 0.955555526081236


  0%|          | 0/7854 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



epoch 5 test acc 0.9491222625413802


In [22]:
torch.cuda.empty_cache()

In [30]:
import torch
import torchvision.models as models


#모델의 형태를 포함하여 저장하기
# torch.save(model, 'KoBERT_QA_v.0.0.2_sona.pth')
# torch.save(model.state_dict(), "./KoBERT_QA_v.0.0.2_sona.pt")

## 3-5.새로운 문장 테스트

In [49]:
#불러오기
# model_pt= BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model_pt.load_state_dict(torch.load('KoBERT_QA_v.0.0.2_sona.pt'))

device = torch.device("cuda")
model_pt = BERTClassifier(bertmodel,  dr_rate=0.5)
model_pt.load_state_dict(torch.load('KoBERT_QA_v.0.0.2_sona.pt'))
model_pt.to(device)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [62]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model_pt.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model_pt(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            # 고객 질문: 1, 상담원 질문: 2, 고객 및 상담원 대답: 0

            print(np.argmax(logits))
#             if np.argmax(logits) == 0:
#                 test_eval.append("대답")
#             elif np.argmax(logits) == 1:
#                 test_eval.append("고객 질문")
#             elif np.argmax(logits) == 2:
#                 test_eval.append("상담원 질문")

#         print(">> 입력하신 내용은 " + test_eval[0] + " 라고 판단됩니다.")

using cached model. /home/adminuser/notebooks/modeling/question/[0차] KoBERT_QA_baseline/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [63]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 이렇게 말하면 몇이 나올까
1


하고싶은 말을 입력해주세요 : 일단은 돌려돌려를 해볼까나
1


하고싶은 말을 입력해주세요 : 몇시까지 하나요
0




KeyboardInterrupt: Interrupted by user